In [ ]:
# Elemental Libraries 
import pandas as pd 
import numpy as np

# Basic Visualization
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 
sns.set_style(style="whitegrid")
import cufflinks as cf
cf.go_offline()

#Other Visualiation
import joypy as jp
from matplotlib import cm
from IPython.display import display
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

# Methodology 
 ### In analzying data, you might find easy to lose the focus, that is why I am following OSEMN methodology 
 which consist on 
 * Obtain Data
 * Scrub Data
 * Explore Data
 * Model Data
 * Interpret Result 


In [ ]:
from PIL import Image
display(Image.open('../input/imagenes/osemn.jpeg'))


# Obtain Data

In [ ]:
df = pd.read_csv('../input/iris-flower-dataset/IRIS.csv')

print("----Techinical Information----")
print('Data Set Shape = {}'.format(df.shape))
print('Data Set Memory Usage = {:.2f} MB'.format(df.memory_usage().sum()/1024**2))
print('\n')
df.describe()

# Scrub Data

## 1. General View

In [ ]:
print('df_columns are ={}'.format(df.columns.to_list()))
print('-------------')
print('Species in data are = {}'.format(df['species'].value_counts()))
print('-------------')
print('Target Column is = {}'.format(df.columns[-1]))

In [ ]:
# 2. Finding Missing Values 
df.isnull().sum()

In [ ]:
# 3. Tranforming Data
from sklearn.preprocessing import OrdinalEncoder

Ordi_Var = df.iloc[:,4:]    # From Text to Numbers using Ordinal Encoder
OE = OrdinalEncoder().fit_transform(Ordi_Var)
df['target'] = OE
df.head()

# Explore Data

## Visualization

In [ ]:
sns.pairplot(data=df, hue='species', palette='Set2')
plt.show()

In [ ]:
label = 'species'

f,axes = plt.subplots(2,2, figsize = (10,10) , dpi=100)
sns.violinplot(x = label , y = 'sepal_length', data = df , ax= axes[0,0])
sns.violinplot(x = label , y = 'sepal_width', data = df , ax= axes[0,1])
sns.violinplot(x = label , y = 'petal_length', data = df , ax= axes[1,0])
sns.violinplot(x = label  , y = 'petal_width', data = df , ax= axes[1,1])
plt.show()

# Well... and what does it mean ?  

***if the violin plot could speak*** , it would say
- *** Iris has two parts 1. Sepal , 2. Petal
- hey bro look this ***"Iris_virgnica"*** has the  biggest sepal in terms of length from 4cm to 8cm but most of them has a 6.5cm lenght average*** No idea what is this... well if the flower were a man , "he would be the fattest man" , no idea yet, so check the image 
- *** Iris Setosa *** is long , think of it as snake it would be the longest snake among those 3 
- *** Iris virginica petal*** is the biggest one... look " from 4 cm to 7 cm , +-3.4 avergae , and also in tems of width 

Basically bro Im trying to say that ***Iris Virginica is the Biggest dude in the Iris neighborhood ***


So when you put some data in the algorithm I will compare all of them and tell you I know wha it that...


In [ ]:
!pip install joypy
import joypy

joypy.joyplot(data=df.iloc[:,0:], by = 'species'  ,overlap=7, figsize=(10,10), legend = True)
plt.show()

1. look this image, the red one means "petal width" Iris-Setosa has the maximun width ,the maximun "petal length" and the maximun "sepal legth"
2.  the orange one means "sepal width" in this case Iris Virginica won

In [ ]:
display(Image.open('../input/imagenes/dont understan.jfif'))

## Check this out

In [ ]:
display(Image.open('../input/imagenes/i_image.jpg'))

### Anomalies and Outliers

Now We have to go through the data set to make sure there are not ***ouliers*** (in this case it means there are not Iris from chernobyl with really long petal or with no petal or any other wierd part) Then we are going to check the relationship between petal and length using heatmap (really usefull when we have many columns "Variable")

In [ ]:
# Analizying Target Variable 
df['species'].value_counts()

# We want to predict or classify Iris (flowers) , therefore we should have a balanced data set
# 3 flowers to predict , with n-rows for each one

In [ ]:
# Dropping columns
df.drop('species',axis = 1, inplace=True)

def corr_heat (frame):   #<---Heat Map
    correlation = frame.corr()
    f,ax = plt.subplots(figsize=(15,10))
    mask = np.triu(correlation)
    sns.heatmap(correlation, annot=True, mask=mask,ax=ax,cmap='viridis')
    bottom,top = ax.get_ylim()
    ax.set_ylim(bottom+ 0.5, top - 0.5)
    
    
corr_heat(df)

### And this what?

Here we can notice that Sepal-length is strong realted with the Petal width-length in other word it means that these dimmension explain each other , ***think of it like the relation between your arms and legs ,you may guess how long are someone arms by looking his/her legs and vice-versa***

however, the Sepalwidth has not positive relation with petal length/width, *** think of how difficult would be guess how long are someone arms by taking a look of their ears only ***

When We have this cases Usually I wil drop the column but instead I will run two escenario having/no having the Columns just to make sure how would it affect the model 

# Model Data 

### Scenario 1 . Keeping "Sepal_Width_cm"

#### Normalizing - Dropping columns, outliers, etc.

In this case We are going to check the all columns to find any outliers

In [ ]:
df.iloc[:,:4].iplot(kind= 'box' , boxpoints = 'outliers')
plt.show()

Here we can notice that ***Sepal_Width_cm has some outliers*** , generally speaking outlier are those values that vary significatly comparing to the others, When we have outlier we have to evaluate if they need to be drop. There are many techniques avalable to Detect and Drop outliers but now I will use the ***Boxplot*** Visualization to find them - all values greater than 4 and lesser than 2.2

In [ ]:
# Extracting Outliers 

#Finding by index
Outliers = df[(df['sepal_width']>4) | (df['sepal_width']<2.2)].index

#Visualization
df[(df['sepal_width']>4) | (df['sepal_width']<2.2)]

In [ ]:
# Dropping 
df.drop(Outliers, axis = 0 , inplace=True)

df.iloc[:,:4].iplot(kind= 'box' , boxpoints = 'outliers')
plt.show()

In [ ]:
# Scenario 1 
SC_X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

# Scenario 2 
SC_1X = df[['sepal_length', 'petal_length', 'petal_width']]

#Variable to predict 
y = df.iloc[:,-1]

# Machine Learning 

1. Logistic Regression
2. Decision Tree Classification
3. Random Forest Classification

In [ ]:
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Scenario 1 

In [ ]:
#---> Splitting Data
X_train, X_test, y_train, y_test = train_test_split(SC_X, y, test_size=0.2, random_state=104)


# 2. Second Bring the model 
LGR = LogisticRegression(random_state=0) #Logistic Regression
DT = DecisionTreeClassifier() #Decision Tree
RDF = RandomForestClassifier () #Randome Forest 


# --------------> 3.Fit models
#-->Scenario 1 
LGR = LGR.fit(X_train,y_train)
DT = DT.fit(X_train,y_train)
RDF = RDF.fit(X_train,y_train)

#4. Prediction
y_pred_LGR = LGR.predict(X_test)
y_pred_DT = DT.predict(X_test)
y_pred_RDF = RDF.predict(X_test)

## Scenario 2

In [ ]:
#---> Splitting Data
X1_train, X1_test, y_train, y_test = train_test_split(SC_1X, y, test_size=0.2, random_state=104)


# 2. Second Bring the model 
LGR1 = LogisticRegression(random_state=0) #Logistic Regression
DT1 = DecisionTreeClassifier() #Decision Tree
RDF1 = RandomForestClassifier () #Randome Forest 


# --------------> 3.Fit models
#-->Scenario 1 
LGR1 = LGR1.fit(X1_train,y_train)
DT1 = DT1.fit(X1_train,y_train)
RDF1 = RDF1.fit(X1_train,y_train)

#4. Prediction
y_pred_LGR1 = LGR1.predict(X1_test)
y_pred_DT1 = DT1.predict(X1_test)
y_pred_RDF1 = RDF1.predict(X1_test)

# Interpret result 

In [ ]:

#Confusion Matriz
print("-------------------------Scenario 1-----------------------")
print("\n")
print("The acurracy score of Logistical regression is {}".format(accuracy_score(y_test,y_pred_LGR)),"\n",confusion_matrix(y_test,y_pred_LGR))
print("\n")
print("The acurracy score of Decision Tree is {}".format(accuracy_score(y_test,y_pred_DT)),"\n",confusion_matrix(y_test,y_pred_DT))
print("\n")
print("The acurracy score of Randome Forest is {}".format(accuracy_score(y_test,y_pred_RDF)),"\n",confusion_matrix(y_test,y_pred_RDF))
print("\n")

In [ ]:

#Confusion Matriz
print("-------------------------Scenario 2-----------------------")
print("\n")
print("The acurracy score of Logistical regression-2nd Scenario is {}".format(accuracy_score(y_test,y_pred_LGR1)),"\n",confusion_matrix(y_test,y_pred_LGR1))
print("\n")
print("The acurracy score of Decision Tree-2nd Scenario is {}".format(accuracy_score(y_test,y_pred_DT1)),"\n",confusion_matrix(y_test,y_pred_DT1))
print("\n")
print("The acurracy score of Randome Forest-2nd Scenario is {}".format(accuracy_score(y_test,y_pred_RDF1)),"\n",confusion_matrix(y_test,y_pred_RDF1))
print("\n")

# Finally... I can conclude

1. All moddel preformance excellent 
2. SepalWidthCm did not affect significatly the performance , in other word having or not does not make any significant improvement 
3. to evaluate the model we can use cross_validation_Score 
4. if we have 30 test, 70 train the model tends to be less accurate 